In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Define your data directory containing subfolders with images
data_dir = r'dt'
validation_data_dir = r"validation"


# Set hyperparameters
img_height = 150
img_width = 150
epochs = 100

In [3]:
from tqdm import tqdm
import cv2
from glob import glob
import pandas as pd
x_data = [] 
y_data = [] 


In [4]:
for category in glob(data_dir+'/*'):
    for file in tqdm(glob(category+'/*')):
        img_array=cv2.imread(file)
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        x_data.append(img_array) 
        y_data.append(category.split("/")[-1])
        
data=pd.DataFrame({'image': x_data,'label': y_data})

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 102/102 [00:00<00:00, 508.39it/s]


In [5]:
data.shape

(710, 2)

In [6]:
from collections import Counter
Counter(y_data)

Counter({'dt\\plastic': 108,
         'dt\\vegetable waste': 102,
         'dt\\aluminium': 100,
         'dt\\cloths': 100,
         'dt\\glass': 100,
         'dt\\iron': 100,
         'dt\\paper': 100})

In [7]:
className = glob(data_dir + '/*' )
numberOfClass = len(className)
print("Number Of Class: ",numberOfClass)

Number Of Class:  7


In [8]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape = (224,224,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(64,(3,3)))  
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Conv2D(128,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(numberOfClass)) # output
model.add(Activation("sigmoid"))

model.compile(loss = "binary_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

batch_size = 256

In [9]:
train_datagen = ImageDataGenerator(rescale= 1./255)

In [10]:
test_datagen = ImageDataGenerator(rescale= 1./255)

In [11]:
train_generator = train_datagen.flow_from_directory(
        data_dir, 
        target_size= (224,224),
        batch_size = batch_size,
        color_mode= "rgb",
        class_mode= "categorical")

test_generator = test_datagen.flow_from_directory(
        validation_data_dir, 
        target_size= (224,224),
        batch_size = batch_size,
        color_mode= "rgb",
        class_mode= "categorical")

Found 710 images belonging to 7 classes.


Found 706 images belonging to 7 classes.


In [16]:
hist = model.fit_generator(
        generator = train_generator,
        epochs=epochs,
        validation_data = test_generator)

C:\Users\darsh\AppData\Local\Temp\ipykernel_18748\1621992094.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/100
3/3 [==============================] - 68s 18s/step - loss: 0.4069 - accuracy: 0.3197 - val_loss: 0.3469 - val_accuracy: 0.4703
Epoch 2/100
3/3 [==============================] - 49s 20s/step - loss: 0.3985 - accuracy: 0.3592 - val_loss: 0.3496 - val_accuracy: 0.4603
Epoch 3/100
3/3 [==============================] - 43s 13s/step - loss: 0.3914 - accuracy: 0.3521 - val_loss: 0.3448 - val_accuracy: 0.4873
Epoch 4/100
3/3 [==============================] - 45s 19s/step - loss: 0.3887 - accuracy: 0.3592 - val_loss: 0.3328 - val_accuracy: 0.4972
Epoch 5/100
3/3 [==============================] - 42s 12s/step - loss: 0.3963 - accuracy: 0.3437 - val_loss: 0.3412 - val_accuracy: 0.4745
Epoch 6/100
3/3 [==============================] - 55s 17s/step - loss: 0.3766 - accuracy: 0.4127 - val_loss: 0.3320 - val_accuracy: 0.4915
Epoch 7/100
3/3 [==============================] - 42s 13s/step - loss: 0.3858 - accuracy: 0.3972 - val_loss: 0.3337 - val_accuracy: 0.5113
Epoch 8/100
3/3 [===

In [17]:
# Print final validation accuracy
final_val_accuracy = hist.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy}")

Final Validation Accuracy: 0.7974504232406616


In [18]:
# Save the trained model
model.save('plant_classification_model.keras')